In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


In [4]:
clean_train_df=pd.read_csv('clean_train.csv')
clean_test_df=pd.read_csv('clean_test.csv')
clean_train_df

,Unnamed: 0,age,worker_class,ind_recode,occ_recode,education,hourly_wage,enrolled_lastwk,marital,ind_code,...,birth_father_nation,birth_mother_nation,self_birth_nation,citizenship,ownbiz_selfempl,VA_questionaire,VAbenefits,wksworked_year,year,50K_label
0,0,1.725879,Not in universe,0,0,High school graduate,0.0,Not in universe,Widowed,Not in universe or children,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0.0,95,0
1,1,1.053560,Self-employed-not incorporated,4,34,Some college but no degree,0.0,Not in universe,Divorced,Construction,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,1.0,94,0
2,2,-0.739291,Not in universe,0,0,10th grade,0.0,High school,Never married,Not in universe or children,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0.0,95,0
3,3,-1.142682,Not in universe,0,0,Children,0.0,Not in universe,Never married,Not in universe or children,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0.0,94,0
4,4,-1.097861,Not in universe,0,0,Children,0.0,Not in universe,Never married,Not in universe or children,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0.0,94,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199518,199518,2.353376,Not in universe,0,0,7th and 8th grade,0.0,Not in universe,Married-civilian spouse present,Not in universe or children,...,Canada,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0.0,95,0
199519,199519,1.367309,Self-employed-incorporated,37,2,11th grade,0.0,Not in universe,Married-civilian spouse present,Business and repair services,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,1.0,94,0
199520,199520,0.560526,Not in universe,0,0,Some college but no degree,0.0,Not in universe,Married-civilian spouse present,Not in universe or children,...,Poland,Poland,Germany,Foreign born- U S citizen by naturalization,0,Not in universe,2,1.0,95,0
199521,199521,-0.828933,Not in universe,0,0,10th grade,0.0,High school,Never married,Not in universe or children,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0.0,95,0


In [5]:
clean_train_df['cap_gains']

0         0.000000
1         0.000000
2         0.000000
3         0.000000
4         0.000000
            ...   
199518    0.000000
199519    0.064181
199520    0.000000
199521    0.000000
199522    0.000000
Name: cap_gains, Length: 199523, dtype: float64

In [6]:
print(clean_train_df['50K_label'].value_counts(),clean_test_df['50K_label'].value_counts())


0    187141
1     12382
Name: 50K_label, dtype: int64 0    93576
1     6186
Name: 50K_label, dtype: int64


Since approximately 93.8% of the label data is in class 0, we are striving for a predictor which yields a greater accuracy.

In [7]:
clean_train_label_vec=clean_train_df['50K_label']
clean_test_label_vec=clean_test_df['50K_label']
clean_train_df.drop(columns='50K_label',inplace=True)
clean_test_df.drop(columns='50K_label',inplace=True)


In [8]:
clean_train_df.drop(columns='Unnamed: 0',inplace=True)
clean_test_df.drop(columns='Unnamed: 0',inplace=True)

In [9]:
clean_test_df=pd.get_dummies(clean_test_df,drop_first=True)
clean_train_df=pd.get_dummies(clean_train_df,drop_first=True)

In [10]:
print(clean_train_df.dtypes)

age                                                         float64
ind_recode                                                    int64
occ_recode                                                    int64
hourly_wage                                                 float64
cap_gains                                                   float64
                                                             ...   
citizenship_ Native- Born abroad of American Parent(s)        uint8
citizenship_ Native- Born in Puerto Rico or U S Outlying      uint8
citizenship_ Native- Born in the United States                uint8
VA_questionaire_ Not in universe                              uint8
VA_questionaire_ Yes                                          uint8
Length: 381, dtype: object


In [11]:
selector  = SelectKBest(f_classif, k=10)
selector.fit(clean_train_df, clean_train_label_vec)
cols_idxs = selector.get_support(indices=True)
train_df_new = clean_train_df.iloc[:,cols_idxs]
train_df_new.columns

Index(['cap_gains', 'num_persons_employer', 'wksworked_year',
       'worker_class_ Not in universe',
       'ind_code_ Not in universe or children',
       'occ_code_ Executive admin and managerial', 'occ_code_ Not in universe',
       'occ_code_ Professional specialty', 'det_family_stat_ Householder',
       'det_household_stat_ Householder'],
      dtype='object')

In [12]:
test_df_new=clean_test_df[['cap_gains', 'num_persons_employer', 'wksworked_year',
       'worker_class_ Not in universe',
       'ind_code_ Not in universe or children',
       'occ_code_ Executive admin and managerial', 'occ_code_ Not in universe',
       'occ_code_ Professional specialty', 'det_family_stat_ Householder',
       'det_household_stat_ Householder']]
test_df_new.columns

Index(['cap_gains', 'num_persons_employer', 'wksworked_year',
       'worker_class_ Not in universe',
       'ind_code_ Not in universe or children',
       'occ_code_ Executive admin and managerial', 'occ_code_ Not in universe',
       'occ_code_ Professional specialty', 'det_family_stat_ Householder',
       'det_household_stat_ Householder'],
      dtype='object')

In [13]:
LR = LogisticRegression()
RF= RandomForestClassifier()
XGBC=XGBClassifier()

In [14]:
#clean_train_label_vec
LR.fit(train_df_new,clean_train_label_vec) 
LR_train_pred=LR.predict(train_df_new) 
LR_test_pred=LR.predict(test_df_new)
print(confusion_matrix(clean_train_label_vec,LR_train_pred),accuracy_score(clean_train_label_vec,LR_train_pred),"\n\n",confusion_matrix(clean_test_label_vec,LR_test_pred),
accuracy_score(clean_test_label_vec,LR_test_pred))

[[186650    491]
 [ 10912   1470]] 0.9428486941355132 

 [[93325   251]
 [ 5484   702]] 0.9425131813716645


In [15]:
RF.fit(train_df_new,clean_train_label_vec) 
RF_train_pred=RF.predict(train_df_new) 
RF_test_pred=RF.predict(test_df_new)
print(confusion_matrix(clean_train_label_vec,RF_train_pred),accuracy_score(clean_train_label_vec,RF_train_pred),"\n\n",confusion_matrix(clean_test_label_vec,RF_test_pred),
accuracy_score(clean_test_label_vec,RF_test_pred))

[[186729    412]
 [ 10138   2244]] 0.9471238904787919 

 [[93259   317]
 [ 5188   998]] 0.9448186684308655


In [16]:
XGBC.fit(train_df_new,clean_train_label_vec) 
XGB_train_pred=XGBC.predict(train_df_new) 
XGB_test_pred=XGBC.predict(test_df_new)
print(confusion_matrix(clean_train_label_vec,XGB_train_pred),accuracy_score(clean_train_label_vec,XGB_train_pred),"\n\n",confusion_matrix(clean_test_label_vec,XGB_test_pred),
accuracy_score(clean_test_label_vec,XGB_test_pred))

[[186783    358]
 [ 10298   2084]] 0.9465926234068253 

 [[93366   210]
 [ 5211   975]] 0.9456606724003127


In [61]:
lr_grid = {'C':[0.001,0.01,0.1,1,10,100],'penalty': ['l1', 'l2','elasticnet', None],'solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag', 'saga']}
lr_grid_search=GridSearchCV(estimator=LR,param_grid=lr_grid,cv=5,verbose=4)
lr_grid_search.fit(train_df_new,clean_train_label_vec)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV 1/5] END ...C=0.001, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ...C=0.001, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ...C=0.001, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ...C=0.001, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ...C=0.001, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.938 total time=   0.1s
[CV 2/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.938 total time=   0.2s
[CV 3/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.938 total time=   0.4s
[CV 4/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.938 total time=   0.2s
[CV 5/5] END C=0.001, penalty=l1, solver=liblinear;, score=0.938 total time=   0.1s
[CV 1/5] END C=0.001, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, penalty=

C:\Users\ethan\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
510 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ethan\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ethan\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\ethan\AppD

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             verbose=4)

In [62]:
lr_grid_search.best_params_

{'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}

In [17]:
lr_tuned = LogisticRegression(C=100,penalty='l1',solver='liblinear')
lr_tuned.fit(train_df_new,clean_train_label_vec) 
lr_tuned_train_pred=lr_tuned.predict(train_df_new) 
lr_tuned_test_pred=lr_tuned.predict(test_df_new)
print(confusion_matrix(lr_tuned_test_pred,clean_test_label_vec),accuracy_score(lr_tuned_test_pred,clean_test_label_vec))

[[93295  5429]
 [  281   757]] 0.942763777791143


In [18]:
print(lr_tuned.coef_,train_df_new.columns)

[[13.32097828  0.60570921  2.41363722  0.01744319 -0.44693014  1.60363388
   0.97088073  1.52244965  0.50020667  0.72849361]] Index(['cap_gains', 'num_persons_employer', 'wksworked_year',
       'worker_class_ Not in universe',
       'ind_code_ Not in universe or children',
       'occ_code_ Executive admin and managerial', 'occ_code_ Not in universe',
       'occ_code_ Professional specialty', 'det_family_stat_ Householder',
       'det_household_stat_ Householder'],
      dtype='object')


Capital gains is the most important feature for the logistic regression model.  For every additional capital gains dollar, the expected change in log-odds is 13.32.

In [77]:
XGB_param_grid = {'min_child_weight': [1, 5],'gamma': [ 1, 1.5, 2],
                  'subsample': [0.6, 1.0],'colsample_bytree': [0.6, 1.0],
 'max_depth': [3, 4, 5],"learning_rate":[0.1, 0.01]}
XGB_grid_search=GridSearchCV(estimator=XGBC,param_grid=XGB_param_grid,cv=5,verbose=4)
XGB_grid_search.fit(train_df_new,clean_train_label_vec)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV 1/5] END colsample_bytree=0.6, gamma=1, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.946 total time=   2.1s
[CV 2/5] END colsample_bytree=0.6, gamma=1, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.945 total time=   2.0s
[CV 3/5] END colsample_bytree=0.6, gamma=1, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.946 total time=   2.2s
[CV 4/5] END colsample_bytree=0.6, gamma=1, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.945 total time=   2.1s
[CV 5/5] END colsample_bytree=0.6, gamma=1, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.945 total time=   1.7s
[CV 1/5] END colsample_bytree=0.6, gamma=1, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=1.0;, score=0.946 total time=   0.9s
[CV 2/5] END colsample_bytree=0.6, gamma=1, learning_rate=0.1, max_depth=3, min

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                   

In [78]:
XGB_grid_search.best_params_

{'colsample_bytree': 1.0,
 'gamma': 1,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 1,
 'subsample': 1.0}

In [19]:
XGB_tuned = XGBClassifier(colsample_bytree=1,gamma=1,learning_rate=.1,max_depth=5,min_child_weight=1,subsample=1,booster='gblinear')
XGB_tuned.fit(train_df_new,clean_train_label_vec) 
XGB_tuned_train_pred=XGB_tuned.predict(train_df_new) 
XGB_tuned_test_pred=XGB_tuned.predict(test_df_new)
print(confusion_matrix(XGB_tuned_test_pred,clean_test_label_vec),accuracy_score(XGB_tuned_test_pred,clean_test_label_vec))

[19:48:32] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "min_child_weight", "subsample" } are not used.

[[93346  5462]
 [  230   724]] 0.9429442072131673


In [86]:
print(XGB_tuned.coef_,train_df_new.columns)

[14.2953     0.0496346 -0.587779  -0.698274  -0.863565   1.4678
 -0.877193   1.33571    0.612155   0.345496 ] Index(['cap_gains', 'num_persons_employer', 'wksworked_year',
       'worker_class_ Not in universe',
       'ind_code_ Not in universe or children',
       'occ_code_ Executive admin and managerial', 'occ_code_ Not in universe',
       'occ_code_ Professional specialty', 'det_family_stat_ Householder',
       'det_household_stat_ Householder'],
      dtype='object')


The five most important predictive features of the Logistic Regression model are, in sequence: Capital gains, weeks worked/ year, Exucutive administration occupational code, Professional specialty occupational code, and Not in universe occupational code.  Four of the five were the same for the XGBoost model.  The one exception is the "Not in universe ind-code" in lieu of weeks worked/year.
The Logistic Regression model makes its predictions based upon a probability that the regression function generates.  If the probability of a label class > .5, the corresponding class is predicted.  
The XGBoost model uses sequential learners, similar to decision trees.  Gradient descent is employed to minimize the errors.

In [87]:
rf_param_grid = {'n_estimators': range(40,100,20),'max_features': ['auto', 'sqrt', 'log2']
                 ,'max_depth' : range(2,10,2),'criterion' :['gini', 'entropy']}
rf_grid_search=GridSearchCV(estimator=RF,param_grid=rf_param_grid,cv=4,verbose=4)
rf_grid_search.fit(train_df_new,clean_train_label_vec)

Fitting 4 folds for each of 72 candidates, totalling 288 fits
[CV 1/4] END criterion=gini, max_depth=2, max_features=auto, n_estimators=40;, score=0.938 total time=   0.7s
[CV 2/4] END criterion=gini, max_depth=2, max_features=auto, n_estimators=40;, score=0.940 total time=   0.6s
[CV 3/4] END criterion=gini, max_depth=2, max_features=auto, n_estimators=40;, score=0.938 total time=   0.7s
[CV 4/4] END criterion=gini, max_depth=2, max_features=auto, n_estimators=40;, score=0.940 total time=   0.7s
[CV 1/4] END criterion=gini, max_depth=2, max_features=auto, n_estimators=60;, score=0.940 total time=   1.1s
[CV 2/4] END criterion=gini, max_depth=2, max_features=auto, n_estimators=60;, score=0.940 total time=   1.0s
[CV 3/4] END criterion=gini, max_depth=2, max_features=auto, n_estimators=60;, score=0.938 total time=   1.0s
[CV 4/4] END criterion=gini, max_depth=2, max_features=auto, n_estimators=60;, score=0.940 total time=   1.1s
[CV 1/4] END criterion=gini, max_depth=2, max_features=aut

GridSearchCV(cv=4, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10, 2),
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': range(40, 100, 20)},
             verbose=4)

In [88]:
rf_grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 80}

In [90]:
rf_tuned = RandomForestClassifier(criterion='gini',max_depth=8,max_features='auto',n_estimators=80)
rf_tuned.fit(train_df_new,clean_train_label_vec) 
RF_tuned_train_pred=rf_tuned.predict(train_df_new) 
RF_tuned_test_pred=rf_tuned.predict(test_df_new)
print(confusion_matrix(RF_tuned_test_pred,clean_test_label_vec),accuracy_score(RF_tuned_test_pred,clean_test_label_vec))

[[93333  5253]
 [  243   933]] 0.9449088831418777
